In [1]:
import os

import openeo
import requests
from requests import Request, Session

In [2]:
SENTINELHUB_BACKEND_URL = "http://127.0.0.1:5000"
SENTINELHUB_USERNAME = os.environ["SH_CLIENT_ID"]
SENTINELHUB_PASSWORD = os.environ["SH_CLIENT_SECRET"]

In [20]:
connection = openeo.connect(SENTINELHUB_BACKEND_URL)
connection.authenticate_basic(SENTINELHUB_USERNAME, SENTINELHUB_PASSWORD)

def send_authenticated_request(request):
    s = Session()
    r = s.send(connection.auth(request).prepare())  
    try:
        r.raise_for_status()
        return r.json()
    except requests.exceptions.JSONDecodeError as e:
        return r.headers
    except requests.exceptions.RequestException as e:
        print(e)
        print(e.response.text)
    

Find collections that can be ordered:

In [4]:
collections = connection.list_collections()
orderable_collections = [collection for collection in collections if collection.get("order:status") == "orderable"]
orderable_collections

[{'description': "PlanetScope is one of the satellite constellation operated by Planet. PlanetScope satellite constellation consists of more than 130 small satellites called Doves. Each Dove satellite is a CubeSat made of three cubic units and thus measures only 10 cm x 10 cm x 30 cm. The satellites are launched in groups, which constantly improves mission's characteristics such as revisit times, spatial and spectral resolutions. The constellation is constantly on and does not require an acquisition planning. PlanetScope is commercial data and has to be ordered by the user",
  'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
   'temporal': {'interval': [['2016-01-01T00:00:00Z', None]]}},
  'id': 'PLANETSCOPE',
  'license': 'various',
  'links': [],
  'order:status': 'orderable',
  'stac_version': '1.0.0'},
 {'description': 'Pleiades is a satellite constellation providing very high-resolution optical imagery and is owned by Airbus. It is now possible to purchase, order and access

In [5]:
collection_to_order = "SPOT"

Get properties we can filter our search by:

In [6]:
requests.get(f"{SENTINELHUB_BACKEND_URL}/collections/{collection_to_order}/queryables").json()

{'$id': 'https://stac-api.example.com/queryables',
 '$schema': 'https://json-schema.org/draft/2019-09/schema',
 'additionalProperties': True,
 'properties': {'expirationDate': {'properties': {'from': {'description': 'ISO-8601 time representing start of search interval, e.g. 2019-01-31T14:00:00+01:00',
     'format': 'date-time',
     'type': 'string'},
    'to': {'description': 'ISO-8601 time representing end of search interval, e.g. 2019-02-05T15:00:00+01:00.',
     'format': 'date-time',
     'type': 'string'}},
   'type': 'object'},
  'maxCloudCoverage': {'default': 100,
   'description': 'The maximum allowable cloud coverage in percent.',
   'format': 'double',
   'maximum': 100,
   'minimum': 0,
   'type': 'number'},
  'maxIncidenceAngle': {'default': 90,
   'description': 'The maximum allowable incidence angle in degrees.',
   'format': 'double',
   'maximum': 90,
   'minimum': 0,
   'type': 'number'},
  'maxSnowCoverage': {'default': 100,
   'description': 'The maximum allowable

Example search for available products:

In [7]:
payload = {
   "bbox":[4.9, 47.9, 5, 48],
   "datetime":"2022-05-20T00:00:00.000Z/2022-08-20T23:59:59.999Z",
   "collections":[collection_to_order],
   "limit":50,
   "filter":{
      "op":"and",
      "args":[
         {
            "op":"=",
            "args":[
               {
                  "property":"maxCloudCoverage"
               },
               67
            ]
         },
         {
            "op":"=",
            "args":[
               {
                  "property":"processingLevel"
               },
               "ALBUM"
            ]
         }
      ]
   }
}

available_products = send_authenticated_request(Request('POST', f"{SENTINELHUB_BACKEND_URL}/search", json=payload))
available_products

{'features': [{'assets': {},
   'geometry': {'coordinates': [[[4.627407610599472, 48.102412163813085],
      [4.633725079743262, 47.56701513374118],
      [5.438156271022335, 47.564157439121544],
      [5.443088162798822, 48.09849582876027],
      [4.627407610599472, 48.102412163813085]]],
    'type': 'Polygon'},
   'id': '784ec28c-7387-4a69-80a9-7764cd737683',
   'links': [],
   'properties': {'acquisitionDate': '2022-06-16T10:14:11.499Z',
    'acquisitionIdentifier': 'DS_SPOT7_202206161014115_GS1_GS1_GS1_GS1_E005N48_01871',
    'acquisitionStation': 'GS1',
    'activityId': 'f57310b9-2174-4474-b41f-9069c5854676',
    'archivingCenter': 'GS1',
    'azimuthAngle': 179.98432747737607,
    'cloudCover': 0,
    'constellation': 'SPOT',
    'correlationId': '1b38b1a7-31c3-4914-aefd-34015ccd6683',
    'expirationDate': '2032-06-27T11:27:06.212530813Z',
    'format': 'image/tiff',
    'geometryCentroid': {'lat': 47.833541741177775, 'lon': 5.035247886698641},
    'id': '784ec28c-7387-4a69-80a

Get list of existing orders:

In [8]:
send_authenticated_request(Request('GET', f"{SENTINELHUB_BACKEND_URL}/orders"))

{'links': {'currentToken': '0'},
 'orders': [{'costs': None,
   'id': '3e664445-8cf0-400c-a81d-b3e77b4c5b1a',
   'items': ['988736cf-50c9-43e2-90bc-33c9675c2dea'],
   'order:date': '2022-08-17T08:25:43.072417Z',
   'order:id': '3e664445-8cf0-400c-a81d-b3e77b4c5b1a',
   'order:status': 'orderable',
   'source_collection_id': 'SPOT',
   'target_collection_id': 'SPOT'},
  {'costs': None,
   'id': '6729269f-6059-4395-9a7f-b57f702928b6',
   'items': ['74f0205c-138f-477a-9b95-b1800661cc1d'],
   'order:date': '2022-08-20T15:56:43.320183Z',
   'order:id': '6729269f-6059-4395-9a7f-b57f702928b6',
   'order:status': 'ordered',
   'source_collection_id': 'SPOT',
   'target_collection_id': 'SPOT'},
  {'costs': None,
   'id': '7418382f-c765-413e-b497-6c8c9426e578',
   'items': ['15b33de6-a301-4270-b240-c965ed0ef268'],
   'order:date': '2022-08-17T09:01:36.253353Z',
   'order:id': '7418382f-c765-413e-b497-6c8c9426e578',
   'order:status': 'orderable',
   'source_collection_id': 'PLEIADES',
   'target

Parameters that can be used in the orders are listed in collection information.

In [25]:
connection.describe_collection(collection_to_order)["order_parameters"]

KeyError: 'order_parameters'

Prepare an order:

In [23]:
payload = {
   "source_collection_id": collection_to_order,
   "items":[
      available_products["features"][0]["id"]
   ],
   "parameters":{
      "geometry": available_products["features"][0]["geometry"]
   }
}

headers = send_authenticated_request(Request('POST', f"{SENTINELHUB_BACKEND_URL}/orders", json=payload))

In [24]:
send_authenticated_request(Request('GET', f"{SENTINELHUB_BACKEND_URL}/orders/{headers['OpenEO-Identifier']}"))

{'costs': None,
 'id': '401805c9-ec10-457b-ba0a-20e9251acea0',
 'items': ['784ec28c-7387-4a69-80a9-7764cd737683'],
 'order:date': '2022-08-22T12:18:50.882335Z',
 'order:id': '401805c9-ec10-457b-ba0a-20e9251acea0',
 'order:status': 'orderable',
 'source_collection_id': 'SPOT',
 'target_collection_id': 'SPOT'}